In [ ]:

#importy koniecznych bibliotek
import pandas as pd
import boto3
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from pycaret.regression import *


ImportError: cannot import name 'load_dotenvy' from 'dotenv' (c:\Users\aleks\miniconda3\envs\od_zera_do_ai\Lib\site-packages\dotenv\__init__.py)

In [ ]:
#ładowanie zmiennych, klucze do łączenia się z bucket w DigitalOcean
load_dotenv()

Umieszczenie danych w Digital Ocean Spaces.

In [ ]:
#tworzenie klienta s3 do kontaktu z chmurą
s3= boto3.client("s3")

In [ ]:
#nazwa bucketu w chmurze
BUCKET_NAME="halfmarathontime"

In [ ]:
#przesłanie plików półmaraton 2023 i 2024 do bucketu w chmurze

s3.upload_file(
    Filename='halfmarathon_wroclaw_2023__final.csv',
    Bucket=BUCKET_NAME,
    Key='stocks/year=2023/halfmarathon_wroclaw_2023__final.csv'
)

s3.upload_file(
    Filename= 'halfmarathon_wroclaw_2024__final.csv',
    Bucket=BUCKET_NAME,
    Key='stocks/year=2024/halfmarathon_wroclaw_2024__final.csv'
)
print(f"Pliki zostały wysłane do Cloud Storage")

Wczytanie danych z Digital Ocean Spaces. 

In [ ]:
#pobrane pliki zostana zapisane w folderze Dane
os.makedirs("Dane", exist_ok=True)
s3.download_file(BUCKET_NAME,"stocks/year=2023/halfmarathon_wroclaw_2023__final.csv", "Dane/halfmarathon_2023.csv")
print(f"Plik zostały pobrany i zapisany.")

In [ ]:
s3.download_file(BUCKET_NAME, "stocks/year=2024/halfmarathon_wroclaw_2024__final.csv", "Dane/halfmarathon_2024.csv")
print(f"Plik zostały pobrany i zapisany.")

EDA i czyszczenie danych. 

In [ ]:
#łączenie danych
file_2023 = "Dane/halfmarathon_2023.csv"
file_2024 = "Dane/halfmarathon_2024.csv"

df_2023 = pd.read_csv(file_2023, sep=";")
df_2024 = pd.read_csv(file_2024, sep=";")

#Połączenie DataFrame'ów w jeden
df_full = pd.concat([df_2023, df_2024], ignore_index=True)

In [ ]:
df_full.head()

In [ ]:
df_full.info()

In [ ]:
# Snippet pomocniczy - zmiana czasu na sekundy

def convert_time_to_seconds(time):
    if pd.isnull(time) or time in ['DNS', 'DNF']:
        return None
    time = time.split(':')
    return int(time[0]) * 3600 + int(time[1]) * 60 + int(time[2])

In [ ]:
df_full['Czas'] = df_full['Czas'].apply(convert_time_to_seconds)

In [ ]:
time_columns = [ '5 km Czas', '10 km Czas', '15 km Czas', '20 km Czas']
for col in time_columns:
    df_full[col] = df_full[col].apply(convert_time_to_seconds)

In [ ]:
df_full.head()

In [ ]:
df_full.info()

In [ ]:
#brakujace wartości
df_full.isnull().sum()

In [ ]:
# Usunięcie wierszy, gdzie brakuje wartości w kolumnie 'Czas'
df_cleaned = df_full.dropna(subset=['Czas'])

In [ ]:
df_cleaned.isnull().sum()

In [ ]:
print("Procent pozostałych danych:", round((len(df_cleaned) / len(df_full)) * 100, 2), "%")

In [ ]:
#dulikaty
df_cleaned[df_cleaned.duplicated()]

In [ ]:
df_cleaned.columns

In [ ]:
# reset kolumn
df_cleaned = df_cleaned[['Płeć', 'Kategoria wiekowa','Rocznik', '5 km Czas', '5 km Tempo', '10 km Czas', '10 km Tempo',
        '15 km Czas', '15 km Tempo', '20 km Czas', '20 km Tempo', 'Czas', 'Tempo']].copy()

In [ ]:
df_cleaned.info()

In [ ]:

#zmiana kolumny Rocznik na Wiek
df_cleaned['Wiek'] = 2024 - df_cleaned['Rocznik']
df_cleaned = df_cleaned.drop(columns=['Rocznik'])

In [ ]:
df_cleaned.info()

In [ ]:
df_cleaned.head()

In [ ]:
df_cleaned.isnull().sum()

In [ ]:
# wartości brakujace w ujęciu procentowym

df_cleaned.isnull().sum() / len(df_cleaned) 

In [ ]:
#usunięcie brakujacych wartości w kolumnie Wiek
df_cleaned = df_cleaned.dropna(subset=["Wiek"])
df_cleaned.sample(5)

In [ ]:
df_cleaned.info()

In [ ]:
df_cleaned.isnull().sum()

In [ ]:
#usunięcie null w Kategoria wiekowa
df_cleaned = df_cleaned.dropna(subset=["Kategoria wiekowa"])

In [ ]:
print("Procent pozostałych danych:", round((len(df_cleaned) / len(df_full)) * 100, 2), "%")

In [ ]:
#wizualizacja kilku zależności
df_cleaned.plot(kind='scatter', x='5 km Tempo', y='Czas')

In [ ]:
df_cleaned.plot(kind='scatter', x='5 km Czas', y='Czas')

In [ ]:
df_cleaned.plot(kind='scatter', x='Tempo', y='Czas')

In [ ]:
df_cleaned.plot(kind='scatter', x='Wiek', y='Czas')

In [ ]:
df_cleaned.hist('Czas')

In [ ]:
df_cleaned.hist('Wiek')

In [ ]:
#wartości odstajace

# Wykres pudełkowy dla kolumn 'Czas', '5 km Tempo' i '5 km Czas'
df_cleaned[['Czas', '5 km Tempo', '5 km Czas', 'Wiek']].plot(kind='box', vert=False, figsize=(10, 5), grid=True)
plt.show()


In [ ]:

# Lista kolumn, w których usuwamu wartości odstające
columns_to_check = ["Czas", "5 km Tempo", "5 km Czas","Wiek"]


df_no_outliers = df_cleaned.copy()

for col in columns_to_check:
    Q1 = df_cleaned[col].quantile(0.25)  # 1. kwartyl (25%)
    Q3 = df_cleaned[col].quantile(0.75)  # 3. kwartyl (75%)
    IQR = Q3 - Q1  # Rozstęp międzykwartylowy

    # Ustalamy dolną i górną granicę (zwykle 1.5*IQR to standardowy próg)
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Usuwamy wartości odstające dla kolumn
    df_no_outliers = df_no_outliers[(df_no_outliers[col] >= lower_bound) & (df_no_outliers[col] <= upper_bound)]

# Aktualizacja df_cleaned po usunięciu wartości odstających
df_cleaned = df_no_outliers.copy()

# Sprawdzenie, ile procent danych usunięto
print(f"Pozostało {round((len(df_cleaned) / len(df_full)) * 100, 2)}% danych.")


In [ ]:
df_cleaned[['Czas', '5 km Tempo', '5 km Czas', 'Wiek']].plot(kind='box', vert=False, figsize=(10, 5), grid=True)

plt.show()

In [ ]:
df_cleaned.isnull().sum()

In [ ]:
df_cleaned.info()

In [ ]:
df_cleaned.tail(6)

In [ ]:
#zapisanie df z wyczyszczonymi danymi
df_cleaned.to_csv("df_cleaned.csv", index=False)

Trenowanie modelu.

In [ ]:
#przygotowanie exp i określenie potrzebnych cech
exp = setup(
    data=df_cleaned, 
    target='Czas',
    categorical_features=['Płeć'], 
    ignore_features=['Kategoria wiekowa','10 km Czas', '10 km Tempo','15 km Czas', '15 km Tempo',
                     '20 km Tempo','Tempo', '20 km Czas'],
    session_id=123,
)

In [ ]:
#dane po uruchomieniu trenowania
exp.dataset_transformed

In [ ]:
#szukanie najlepszego modelu, MAE metryka, która mierzy średni błąd bezwzględny, (niskie wartości oczekiwane)
best_model = exp.compare_models(sort='MAE')

In [ ]:
metryki_przed_tuningiem = pull()
metryki_przed_tuningiem

In [ ]:
# próba strojenia hiperparametrów, tuning

best_tuned_model = exp.tune_model(best_model, n_iter=200, optimize='MAE')

In [ ]:
#porównanie
best_final_model = compare_models([best_model, best_tuned_model])

In [ ]:
#uruchomienie modelu na holdout set
exp.predict_model(best_final_model)

In [ ]:
# wizualizacja istotności cech

plot_model(best_final_model, plot='feature')

In [ ]:
plot_model(best_final_model, plot="residuals")

In [ ]:

#jak model sie myli
plot_model(best_final_model, plot='error')

In [ ]:
# trenuję model na całym Dataset

final_model = finalize_model(best_final_model)

In [ ]:
final_model

Zapisywanie modelu.

In [ ]:
save_model(final_model, 'huber_model_halfmarathon_time')

In [ ]:
# zapisanie modelu w Spaces Object Storage

s3.upload_file(
    Filename="huber_model_halfmarathon_time.pkl",
    Bucket=BUCKET_NAME,
    Key='stocks/model/huber_model_halfmarathon_time.pkl'
)

Próba działania modelu.

In [ ]:
user_data_df = pd.DataFrame(
        {'Wiek' : 28,
        'Płeć' : 'M',
        '5 km Tempo' : 4.45, 
        '5 km Czas' : 1500,
        }, index=[0])

In [ ]:
import datetime

prediction = predict_model(final_model, data=user_data_df)

prediction_seconds = round(prediction["prediction_label"][0], 2)

prediction_time = str(datetime.timedelta(seconds=int(prediction_seconds)))

print(f'Próba modelu, przewidywanie czas ukończenia półmaratonu: H:M:S wynosi {prediction_time}')

In [ ]:
user_1_data_df = pd.DataFrame(
        {'Wiek' : 32,
        'Płeć' : 'K',
        '5 km Tempo' : 6.45, 
        '5 km Czas' : 2100,
        }, index=[0])

In [ ]:
prediction = predict_model(final_model, data=user_1_data_df)

prediction_seconds = round(prediction["prediction_label"][0], 2)

prediction_time = str(datetime.timedelta(seconds=int(prediction_seconds)))

print(f'Próba modelu, przewidywanie czas ukończenia półmaratonu: H:M:S wynosi {prediction_time}')

In [ ]:
user_2_data_df = pd.DataFrame(
        {'Wiek' : 53,
        'Płeć' : 'M',
        '5 km Tempo' : 5.45, 
        '5 km Czas' : 1900,
        }, index=[0])

In [ ]:

prediction = predict_model(final_model, data=user_2_data_df)

# Pobranie przewidywanej wartości i zaokrąglenie
predicted_seconds = round(prediction["prediction_label"].iloc[0], 2)

# Konwersja sekund na format godzinowy (H:M:S)
predicted_time = str(datetime.timedelta(seconds=int(predicted_seconds)))


print(f"Przewidywany czas ukończenia półmaratonu: {predicted_time}")


In [ ]:
import pandas as pd
df_spr= pd.read_csv("df_cleaned.csv")


In [ ]:
print(df_spr["5 km Tempo"].describe())

In [ ]:
print(df_spr["5 km Czas"].describe())